In [1]:
import tensorflow as tf
import numpy as np
import pprint

import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

from data import WN18
from model.metrics import evaluate_rank

In [2]:
class Config:
    seed = 21
    n_epochs = 10
    batch_size = 128
    embed_dim = 150

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""

def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    
    triples_all = triples_tr + triples_va + triples_te
    
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}

    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)

    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    
    return e2idx, p2idx, params


def build_train_data(triples_tr, e2idx, p2idx):
    x_s = np.array([e2idx[s] for (s, p, o) in triples_tr], dtype=np.int32)
    x_p = np.array([p2idx[p] for (s, p, o) in triples_tr], dtype=np.int32)
    x_o = np.array([e2idx[o] for (s, p, o) in triples_tr], dtype=np.int32)

    x = {'s': x_s,
         'p': x_p,
         'o': x_o}
    y = np.ones([len(x_s)], dtype=np.float32)
    
    return x, y


def train_input_fn(triples_tr, e2idx, p2idx, random_state, params):
    x, y = build_train_data(triples_tr, e2idx, p2idx)
    s, p, o = x['s'], x['p'], x['o']
    
    s_ = random_state.choice(params['e_vocab_size'], s.shape)
    o_ = random_state.choice(params['e_vocab_size'], o.shape)
    
    x_ = {
        's': np.concatenate([s, s_, s]),
        'p': np.concatenate([p, p, p]),
        'o': np.concatenate([o, o, o_])}
    y_ = np.concatenate([y, np.zeros([2*len(y)], dtype=np.float32)])
    
    return tf.estimator.inputs.numpy_input_fn(x = x_,
                                              y = y_,
                                              batch_size = Config.batch_size,
                                              num_epochs = 1,
                                              shuffle = True)

In [4]:
def forward(features, params):
    e_embed = tf.get_variable('e_embed',
                              [params['e_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    p_embed = tf.get_variable('p_embed',
                              [params['p_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    o = tf.nn.embedding_lookup(e_embed, features['o'])
    
    logits = tf.reduce_sum(s*p*o, axis=1)
    
    return logits
    
    
def model_fn(features, labels, mode, params):
    logits = forward(features, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
        
        loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                                         labels=labels))
        
        train_op = tf.train.AdamOptimizer().minimize(
            loss_op, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = loss_op,
                                          train_op = train_op)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = tf.sigmoid(logits))

In [5]:
random_state = np.random.RandomState(Config.seed)
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)

model = tf.estimator.Estimator(model_fn,
                               params = params)

for _ in range(Config.n_epochs):
    model.train(train_input_fn(triples_tr,
                               e2idx,
                               p2idx,
                               random_state,
                               params))
evaluate_rank(model,
              triples_va,
              triples_te,
              triples_all,
              e2idx,
              p2idx,
              params['e_vocab_size'],
              batch_size = Config.batch_size)

Files Already Downloaded
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqv330sxx', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11fe658d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorfl

INFO:tensorflow:loss = 0.70648056, step = 6117 (3.239 sec)
INFO:tensorflow:global_step/sec: 31.2844
INFO:tensorflow:loss = 0.75363207, step = 6217 (3.196 sec)
INFO:tensorflow:global_step/sec: 29.3325
INFO:tensorflow:loss = 0.70683277, step = 6317 (3.409 sec)
INFO:tensorflow:global_step/sec: 28.6847
INFO:tensorflow:loss = 0.6982405, step = 6417 (3.486 sec)
INFO:tensorflow:global_step/sec: 31.5956
INFO:tensorflow:loss = 0.69434655, step = 6517 (3.165 sec)
INFO:tensorflow:global_step/sec: 31.5615
INFO:tensorflow:loss = 0.7173288, step = 6617 (3.168 sec)
INFO:tensorflow:Saving checkpoints for 6632 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqv330sxx/model.ckpt.
INFO:tensorflow:Loss for final step: 0.58971834.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensor

INFO:tensorflow:global_step/sec: 32.522
INFO:tensorflow:loss = 0.20818546, step = 12749 (3.075 sec)
INFO:tensorflow:global_step/sec: 34.3924
INFO:tensorflow:loss = 0.19709477, step = 12849 (2.908 sec)
INFO:tensorflow:global_step/sec: 35.7078
INFO:tensorflow:loss = 0.15861467, step = 12949 (2.800 sec)
INFO:tensorflow:global_step/sec: 34.2208
INFO:tensorflow:loss = 0.14616615, step = 13049 (2.922 sec)
INFO:tensorflow:global_step/sec: 34.4359
INFO:tensorflow:loss = 0.13032958, step = 13149 (2.904 sec)
INFO:tensorflow:global_step/sec: 32.9987
INFO:tensorflow:loss = 0.11769578, step = 13249 (3.030 sec)
INFO:tensorflow:Saving checkpoints for 13264 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqv330sxx/model.ckpt.
INFO:tensorflow:Loss for final step: 0.12925272.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:t

INFO:tensorflow:loss = 0.01206025, step = 19181 (3.570 sec)
INFO:tensorflow:global_step/sec: 28.5334
INFO:tensorflow:loss = 0.0076926313, step = 19281 (3.504 sec)
INFO:tensorflow:global_step/sec: 27.7875
INFO:tensorflow:loss = 0.017924888, step = 19381 (3.599 sec)
INFO:tensorflow:global_step/sec: 27.932
INFO:tensorflow:loss = 0.0066367127, step = 19481 (3.580 sec)
INFO:tensorflow:global_step/sec: 28.5676
INFO:tensorflow:loss = 0.08061032, step = 19581 (3.501 sec)
INFO:tensorflow:global_step/sec: 27.8765
INFO:tensorflow:loss = 0.0367143, step = 19681 (3.587 sec)
INFO:tensorflow:global_step/sec: 32.8433
INFO:tensorflow:loss = 0.036204606, step = 19781 (3.045 sec)
INFO:tensorflow:global_step/sec: 32.651
INFO:tensorflow:loss = 0.01507787, step = 19881 (3.063 sec)
INFO:tensorflow:Saving checkpoints for 19896 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqv330sxx/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0022415235.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.

INFO:tensorflow:global_step/sec: 28.1393
INFO:tensorflow:loss = 0.0022332182, step = 25613 (3.554 sec)
INFO:tensorflow:global_step/sec: 22.6242
INFO:tensorflow:loss = 0.0026911148, step = 25713 (4.420 sec)
INFO:tensorflow:global_step/sec: 28.7563
INFO:tensorflow:loss = 0.002793449, step = 25813 (3.479 sec)
INFO:tensorflow:global_step/sec: 31.7813
INFO:tensorflow:loss = 0.0011144746, step = 25913 (3.145 sec)
INFO:tensorflow:global_step/sec: 28.669
INFO:tensorflow:loss = 0.0016813427, step = 26013 (3.489 sec)
INFO:tensorflow:global_step/sec: 25.122
INFO:tensorflow:loss = 0.00083891395, step = 26113 (3.980 sec)
INFO:tensorflow:global_step/sec: 25.3571
INFO:tensorflow:loss = 0.00043805697, step = 26213 (3.944 sec)
INFO:tensorflow:global_step/sec: 29.2703
INFO:tensorflow:loss = 0.048250716, step = 26313 (3.416 sec)
INFO:tensorflow:global_step/sec: 32.081
INFO:tensorflow:loss = 0.0072523206, step = 26413 (3.117 sec)
INFO:tensorflow:global_step/sec: 33.442
INFO:tensorflow:loss = 0.0069862995,

INFO:tensorflow:loss = 0.000408974, step = 31945 (3.682 sec)
INFO:tensorflow:global_step/sec: 29.5858
INFO:tensorflow:loss = 0.003924832, step = 32045 (3.379 sec)
INFO:tensorflow:global_step/sec: 28.5504
INFO:tensorflow:loss = 0.0008861715, step = 32145 (3.503 sec)
INFO:tensorflow:global_step/sec: 25.3387
INFO:tensorflow:loss = 0.0013574759, step = 32245 (3.946 sec)
INFO:tensorflow:global_step/sec: 24.7176
INFO:tensorflow:loss = 0.00019240147, step = 32345 (4.046 sec)
INFO:tensorflow:global_step/sec: 26.2328
INFO:tensorflow:loss = 0.00053097494, step = 32445 (3.812 sec)
INFO:tensorflow:global_step/sec: 29.9487
INFO:tensorflow:loss = 0.08977248, step = 32545 (3.339 sec)
INFO:tensorflow:global_step/sec: 25.5441
INFO:tensorflow:loss = 0.0005292635, step = 32645 (3.915 sec)
INFO:tensorflow:global_step/sec: 25.109
INFO:tensorflow:loss = 0.0013270996, step = 32745 (3.983 sec)
INFO:tensorflow:global_step/sec: 24.0359
INFO:tensorflow:loss = 0.00031684342, step = 32845 (4.161 sec)
INFO:tensorfl